In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

kevinwtan_gameresults_path = kagglehub.dataset_download('kevinwtan/gameresults')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gameresults/data.csv


In [ ]:
!pip install torchmetrics
import torch
import torch.nn as nn
import torchvision
import torchmetrics
import numpy as np
from tqdm import tqdm
import sklearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.0 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-1

In [ ]:
game_data = pd.read_csv('/kaggle/input/gameresults/data.csv')
game_data.head()

,1,1.1,1924,1.2,1.3,10,0,0.1,0.2,1.4,...,-1.9,1.16,1.17,1.18,4788,False.36,False.37,False.38,False.39,True.9
0,1,1,2474,0,0,10,0,0,0,1,...,-1,1,1,0,5203,False,False,False,False,True
1,0,1,2629,0,1,4,0,0,0,0,...,-1,0,0,1,4688,False,False,False,False,True
2,1,1,1837,0,1,10,0,0,0,2,...,-1,0,0,0,4739,False,False,False,False,True
3,0,1,3043,0,1,3,0,0,0,0,...,-1,0,0,1,4697,False,False,False,False,True
4,1,1,2546,0,0,11,0,0,0,1,...,-1,1,0,0,4995,False,False,False,False,True


In [ ]:
game_data = game_data.map(lambda x: int(x) if isinstance(x, bool) else x)

In [ ]:
X = game_data.iloc[:, 1:]
y = game_data.iloc[:, [0]]

from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X)
X = scaler.transform(X)

# X_tensor.shape, y_tensor.shape

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.float32)

X_tensor.shape, y_tensor.shape

(torch.Size([54448, 236]), torch.Size([54448, 1]))

In [ ]:
# from sklearn import preprocessing
# scaler = preprocessing.StandardScaler().fit(X)
# X = scaler.transform(X)

# X_tensor.shape, y_tensor.shape

(torch.Size([54448, 236]), torch.Size([54448, 1]))

In [ ]:
n_samples = X_tensor.size(0)

train_size = int(0.8 * n_samples)
test_size = n_samples - train_size

from torch.utils.data import random_split, TensorDataset

dataset = TensorDataset(X_tensor, y_tensor)

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
def calc_accuracy_improved(model, dataloader):
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)

            z = model(X)

            predictions = torch.sigmoid(z).squeeze() > 0.5

            predictions = predictions.long()

            y = y.squeeze()

            correct += (predictions == y).sum().item()
            total += y.size(0)

    accuracy = correct / total
    # print(f"Correct {correct}, Total {total}, Accuracy: {accuracy * 100:.2f}%")
    return accuracy


In [ ]:
def evaluation(data, model):
    accuracy = torchmetrics.Accuracy(task="binary").to(device)
    precision = torchmetrics.Precision(task="binary", average="macro").to(device)
    recall = torchmetrics.Recall(task="binary", average="macro").to(device)
    f1 = torchmetrics.F1Score(task="binary", average="macro").to(device)

    accuracy.reset()
    precision.reset()
    recall.reset()
    f1.reset()

    for x_batch, y_batch in data:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        logits = model(x_batch)

        probabilities = torch.sigmoid(logits).squeeze()  # Shape will be [batch_size, 1] -> [batch_size]

        y_pred = (probabilities > 0.5).long()  # Convert boolean to long

        y_batch = y_batch.squeeze()

        accuracy(y_pred, y_batch)
        precision(y_pred, y_batch)
        recall(y_pred, y_batch)
        f1(y_pred, y_batch)

    macro_acc = accuracy.compute()
    macro_prec = precision.compute()
    macro_rec = recall.compute()
    macro_f1 = f1.compute()

    print(f"Macro Accuracy: {macro_acc:.4f}")
    print(f"Macro Precision: {macro_prec:.4f}")
    print(f"Macro Recall: {macro_rec:.4f}")
    print(f"Macro F1 Score: {macro_f1:.4f}")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
nn_model = torch.nn.Sequential(
    torch.nn.Linear(236, 128),
    torch.nn.ReLU(),
    torch.nn.Linear(128, 128),
    torch.nn.ReLU(),
    torch.nn.Linear(128, 64),
    torch.nn.ReLU(),
    torch.nn.Linear(64, 64),
    torch.nn.ReLU(),
    torch.nn.Linear(64, 1)
).to(device)

lr = 1e-5
wd = 1e-3
epochs = 20

improved_nn_model = nn_model

loss_fn = torch.nn.BCEWithLogitsLoss()
opt = torch.optim.Adam(improved_nn_model.parameters(), lr=lr, weight_decay=wd)

train_mses = []
test_mses = []

best_test_mse = np.inf

for epoch in range(epochs):
    improved_nn_model.train()
    for X_vals, y_vals in train_loader:

        X_vals, y_vals = X_vals.to(device), y_vals.to(device)
        opt.zero_grad()

        z = improved_nn_model(X_vals)

        # print(X_vals[0])

        loss = loss_fn(z, y_vals)

        loss.backward()
        opt.step()

    # print(f'epoch {epoch}: loss is {loss.item()}')
    improved_nn_model.eval()
    # with torch.no_grad():
    #     total_mse = 0
    #     for X_vals, y_vals in test_loader:
    #         X_vals, y_vals = X_vals.to(device), y_vals.to(device)
    #         y_vals = y_vals.float().unsqueeze(1)
    #         preds = improved_nn_model(X_vals)

    #         test_mse = loss_fn(preds, y_vals).item()
    #         total_mse += test_mse

    #     avg_mse = total_mse / len(test_loader)

    #     if avg_mse < best_test_mse:
    #         best_test_mse = avg_mse
    #         torch.save(improved_nn_model.state_dict(), 'best_model.ckpt')
    #         print(f'Saved new best model at epoch {epoch} with MSE: {avg_mse:.4f}')

    train_acc = calc_accuracy_improved(improved_nn_model, train_loader)
    test_acc = calc_accuracy_improved(improved_nn_model, test_loader)
    print(f'epoch {epoch} - train acc: {train_acc * 100:.2f}, test acc: {test_acc * 100:.2f}')

# improved_nn_model.load_state_dict(torch.load('best.ckpt', weights_only=True))
# improved_nn_model.load_state_dict(torch.load('best.ckpt', weights_only=True))
# train_acc = calc_accuracy_improved(improved_nn_model, train_loader)
# test_acc = calc_accuracy_improved(improved_nn_model, test_loader)
# print(f'Best - train acc: {train_acc}, test acc: {test_acc}')

evaluation(test_loader, improved_nn_model)

torch.save(improved_nn_model.state_dict(), 'final_nn_model.pth')

epoch 0 - train acc: 96.53, test acc: 96.54
epoch 1 - train acc: 97.27, test acc: 97.24
epoch 2 - train acc: 97.52, test acc: 97.47
epoch 3 - train acc: 97.67, test acc: 97.59
epoch 4 - train acc: 97.83, test acc: 97.85
epoch 5 - train acc: 97.98, test acc: 97.93
epoch 6 - train acc: 98.08, test acc: 98.09
epoch 7 - train acc: 98.19, test acc: 98.12
epoch 8 - train acc: 98.25, test acc: 98.23
epoch 9 - train acc: 98.34, test acc: 98.34
epoch 10 - train acc: 98.40, test acc: 98.37
epoch 11 - train acc: 98.46, test acc: 98.41
epoch 12 - train acc: 98.47, test acc: 98.44
epoch 13 - train acc: 98.57, test acc: 98.54
epoch 14 - train acc: 98.60, test acc: 98.60
epoch 15 - train acc: 98.68, test acc: 98.62
epoch 16 - train acc: 98.71, test acc: 98.63
epoch 17 - train acc: 98.75, test acc: 98.62
epoch 18 - train acc: 98.80, test acc: 98.62
epoch 19 - train acc: 98.82, test acc: 98.67
Macro Accuracy: 0.9867
Macro Precision: 0.9872
Macro Recall: 0.9880
Macro F1 Score: 0.9876


In [ ]:
improved_nn_model.load_state_dict(torch.load('best_model.ckpt'))
improved_nn_model.eval()

In [ ]:
train_acc = calc_accuracy_improved(improved_nn_model, train_loader)
test_acc = calc_accuracy_improved(improved_nn_model, test_loader)
print(f'Best - train acc: {train_acc:.4f}, test acc: {test_acc:.4f}')

evaluation(test_loader, improved_nn_model)